In [ ]:
import numpy as np
import pyvista as pv
import torch
import matplotlib.pyplot as plt
import sys
sys.path.append("./")  # 确保根目录在 sys.path 中
import src.simulator as simulator

In [ ]:
film = np.zeros((70, 100, 150))

bottom = 10
# film[:, :, 0:bottom] = 10 # bottom

height = 80
# left_side = 75
# right_side = 75
# film[:, left_side+6:200-left_side-6, 0:height] = 10
film[:, :19, 0:height] = 10
film[:, 81:, 0:height] = 10

# film = slide2D_fractionZ(film=film, start=[0, left_side, bottom], end=[70, left_side+6, height], direction='y', fraction='+', value=10)
# film = slide2D_fractionZ(film=film, start=[0, 200-left_side-1, bottom], end=[70, 200-left_side-6-1, height], direction='y', fraction='+', value=10)
# film = slide2D_fractionZ(film=film, start=[0, 19+6-1, bottom], end=[70, 19-1, height], direction='y', fraction='+', value=10)
# film = slide2D_fractionZ(film=film, start=[0, 181-6, bottom], end=[70, 181, height], direction='y', fraction='+', value=10)

# film[:, 80:121, 0:31] = 10

film[:, :, 0:bottom] = 10 # bottom
film[:, :, height:] = 0 # bottom

In [ ]:
geom = pv.Box()

substrute = torch.Tensor(film==10).to_sparse()
substrute = substrute.indices().numpy().T

cyan = torch.Tensor(np.logical_and(film[:, :, :]!=10, film[:, :, :]!=0)).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray')
# p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()


In [ ]:
etchfilm = np.zeros((70, 100, 150, 2))
etchfilm[:, :, :, 0] = film
# etchfilm[:, :, :, 1] = film

center = 50

In [ ]:
from scipy import interpolate

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
yield_hist = np.array([[1.0, 1.01, 1.05,  1.2,  1.4,  1.5, 1.07, 0.65, 0.28, 0.08,  0, \
                        0.08, 0.28,0.65,  1.07, 1.5, 1.4, 1.2, 1.05, 1.01, 1.0 ], \
                        [  0,  5,   10,   20,   30,   40,   50,   60,   70,   80, 90, \
                          100, 110, 120, 130, 140, 150, 160, 170, 175, 180]])
yield_hist[1] *= np.pi/180
yield_func = interpolate.interp1d(yield_hist[1], yield_hist[0], kind='quadratic')

xnew = np.arange(0, np.pi/2, 0.01)
# xnew = np.array([])
ynew = yield_func(xnew)   # use interpolation function returned by `interp1d`
plt.plot(yield_hist[1, :11], yield_hist[0, :11], 'o', xnew, ynew, '-')
plt.show()

In [ ]:
logname = 'Multi_species_benchmark_1023_redepo'
inputMethod='bunch'

etchingPoint = np.array([center, center, bottom-30])
depoPoint = np.array([center, center, bottom-30])
density = 10
center_with_direction=np.array([[int(etchfilm.shape[0]/2),int(etchfilm.shape[1]/2),150]])
range3D=np.array([[0, etchfilm.shape[0], 0, etchfilm.shape[1], 0, etchfilm.shape[2]]])
InOrOut=[1]
# yield_hist=np.array([None])
yield_hist = yield_hist
maskTop=40, 
maskBottom=98, 
maskStep=10, 
maskCenter=[int(etchfilm.shape[0]/2), int(etchfilm.shape[1]/2)]
backup=False
mirrorGap=5
offset_distence = 0.8
reaction_type=False
param = [1.6, -0.7]
n=1
celllength=1e-5
kdtreeN=2
filmKDTree=np.array([[2, 0, 1], [3, 0, -1]]) # 1 for depo -1 for etching
# filmKDTree=np.array([[2, 1], [3, 1]])
weightDepo=0.2
weightEtching = 0.2
tstep=1e-5
substrateTop=bottom
posGeneratorType='benchmark'
testEtch = simulator.etching(
                    inputMethod,
                    etchingPoint,depoPoint,
                    density, center_with_direction, 
                    range3D, InOrOut, yield_hist,
                    maskTop, maskBottom, maskStep, maskCenter,backup, 
                    mirrorGap,offset_distence,
                    reaction_type, param,n,
                    celllength, kdtreeN, filmKDTree,weightDepo,weightEtching, tstep,
                    substrateTop,posGeneratorType, logname)

In [ ]:
# inputMethod='bunch'
# etchingPoint = np.array([center, center, 125])
# depoPoint = np.array([center, center, 125])
# density = 10
# center_with_direction=np.array([[int(etchfilm.shape[0]/2),int(etchfilm.shape[1]/2),150]])
# range3D=np.array([[0, etchfilm.shape[0], 0, etchfilm.shape[1], 0, etchfilm.shape[2]]])
# InOrOut=[1]
# # yield_hist=np.array([None])
# yield_hist = yield_hist
# maskTop=40, 
# maskBottom=98, 
# maskStep=10, 
# maskCenter=[int(etchfilm.shape[0]/2), int(etchfilm.shape[1]/2)]
# backup=False
# mirrorGap=5
# offset_distence = 0.8
# reaction_type=False
# param = [1.6, -0.7]
# n=1
# celllength=1e-5
# kdtreeN=5
# filmKDTree=np.array([[2, 0, 1], [3, 0, -1]]) # 1 for depo -1 for etching
# # filmKDTree=np.array([[2, 1], [3, 1]])
# weightDepo=0.2
# weightEtching = 0.2
# tstep=1e-5
# substrateTop=80
# posGeneratorType='top'
# testEtch = simulator.etching(
#                     inputMethod,
#                     etchingPoint,depoPoint,
#                     density, center_with_direction, 
#                     range3D, InOrOut, yield_hist,
#                     maskTop, maskBottom, maskStep, maskCenter,backup, 
#                     mirrorGap, offset_distence,
#                     reaction_type, param,n,
#                     celllength, kdtreeN, filmKDTree,weightDepo,weightEtching, tstep,
#                     substrateTop,posGeneratorType, logname)

In [ ]:
# react_table = np.array([[[0.0, 0, 1], [0.0, 0, 1]],
#                         [[0.0, -1, 0], [1.0, 0, -1]]])

# react_type_table = np.array([[2, 0],
#                              [3, 0]])

In [ ]:
# simulator.react_table = react_table
# simulator.react_type_table = react_type_table

In [ ]:
cicle = 100
celllength=1e-5
parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0.2, 0]])
step1 = testEtch.inputParticle(etchfilm, parcel,'etching', 'benchmark',0, 1, int(1e4), int(1e5), int(5e4),2, 10, 100)

In [ ]:
pos1e4_data = testEtch.planes
point_cloud = pv.PolyData(pos1e4_data[:, 3:6])
vectors = pos1e4_data[:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

# Display the arrowscyan


plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
geom = pv.Box()

substrute = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]>0, etchfilm[:, :, :,0]<1)).to_sparse()
substrute = substrute.indices().numpy().T

# cyan = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]!=0, film[:, :, :]!=10)).to_sparse()
cyan = torch.Tensor(etchfilm[:, :, :,0]>=9).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
# p.add_mesh(subglyphed, color='gray')
p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etchfilm.shape

In [ ]:
geom = pv.Box()

substrute = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]>=9, film[:, :, :]==10)).to_sparse()
substrute = substrute.indices().numpy().T

cyan = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]>=9, film[:, :, :]==0)).to_sparse()
# cyan = torch.Tensor(etchfilm[:, :, :,0]>=9).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray')
p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
np.save('redepo_benchmark4', etchfilm)

In [ ]:
etchfilm =np.load('redepo_benchmark1.npz')

In [ ]:
etchfilm = etchfilm['arr_0']

In [ ]:
geom = pv.Box()

substrute = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]>0, etchfilm[:, :, :,0]<1)).to_sparse()
substrute = substrute.indices().numpy().T

# cyan = torch.Tensor(np.logical_and(etchfilm[:, :, :,0]!=0, film[:, :, :]!=10)).to_sparse()
cyan = torch.Tensor(etchfilm[:, :, :,0]>=9).to_sparse()
cyan = cyan.indices().numpy().T

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

cyanmesh = pv.PolyData(cyan)
cyanmesh["radius"] = np.ones(cyan.shape[0])*0.5
# Progress bar is a new feature on master branch
cyanglyphed = cyanmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)0
p = pv.Plotter()
p.add_mesh(subglyphed, color='gray')
p.add_mesh(cyanglyphed, color='cyan')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
def Rn_coeffcient(c1, c2, c3, c4, alpha):
    return c1 + c2*np.tanh(c3*alpha - c4)

In [ ]:
rn = Rn_coeffcient(0.9423, 0.9434, 2.342, 3.026, np.pi/2)

print(rn)

In [ ]:
# xnew = np.arange(0, np.pi/2, 0.01)
xnew = np.arange(0, np.pi/2, 0.1)
# xnew = np.array([])
ynew = [Rn_coeffcient(0.9423, 0.9434, 2.342, 3.026, i) for i in xnew]  # use interpolation function returned by `interp1d`
ynew /= ynew[-1]
plt.plot(xnew, ynew, '-')
plt.show()

In [ ]:
rn_func = interpolate.interp1d(xnew, ynew, kind='quadratic')
rn_prob = rn_func(1.2)
print(rn_prob)

In [ ]:
a = np.array([1, 0, 0])
b = np.array([-1, 0, 0])
dot_product = np.dot(a, b)
angle_rad = np.arccos(np.clip(dot_product, -1.0, 1.0))
if dot_product < 0:
    angle_rad = np.pi - angle_rad
print(angle_rad)
react_rate = rn_func(angle_rad)

In [ ]:
a = np.array([1, 0.00, 0])
b = np.array([-0.94, 0.06, 0])
dot_product = np.dot(a, b)
print(dot_product)
dot_product = np.abs(dot_product)
print(dot_product)
angle_rad = np.arccos(np.clip(dot_product, -1.0, 1.0))
# if dot_product < 0:
#     angle_rad = np.pi - angle_rad
print(angle_rad)
react_rate = rn_func(angle_rad)